In [ ]:
pip install openai-whisper torch sentence-transformers


In [ ]:
pip show torch


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [ ]:
!pip install google-generativeai tensorflow-addons tensorflow python-dotenv pytube

In [17]:
pip install ffmpeg

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6138 sha256=eeee111b3b423d2221fc9af8de08fa364616c5bb017b06a4e74b6a26ebd950f8
  Stored in directory: c:\users\ant pc\appdata\local\pip\cache\wheels\26\21\0c\c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
Successfully built ffmpeg


In [17]:
pip install Flask python-dotenv google-generativeai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
print(sys.executable)

C:\anaconda3\python.exe


In [3]:
!where python

C:\anaconda3\python.exe
C:\Program Files\Python310\python.exe
C:\Users\ANT pc\AppData\Local\Microsoft\WindowsApps\python.exe


In [3]:
import os
import time
from dotenv import load_dotenv
import google.generativeai as genai
import re

# Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")
# Configure the Gemini API
genai.configure(api_key=api_key)

# Function to split the document into smaller parts
def split_document(document_file_path, max_length):
    with open(document_file_path, 'r', encoding='utf-8') as file:  # Specify encoding
        content = file.read()

    # Split the content into parts based on max_length
    parts = []
    while len(content) > max_length:
        # Find the last space within the max_length limit
        split_index = content.rfind(' ', 0, max_length)
        if split_index == -1:  # No space found, split at max_length
            split_index = max_length
        parts.append(content[:split_index])
        content = content[split_index:].strip()  # Remove leading spaces for the next part

    if content:  # Add the remaining part
        parts.append(content)

    return parts

# Function to transcribe document with segmented chapters
def transcribe_document_with_segmented_chapters(document_file_path):
    # Change the model to Gemini 1.5 Flash-8B
    model = genai.GenerativeModel('models/gemini-1.5-flash-8b')  # Updated to use the Flash model
    prompt = """Please transcribe this document file and segment the transcription into meaningful chapters. 
    For each chapter, provide 5-6 detailed bullet points that summarize the key ideas, themes, or events discussed in that chapter. 
    Do not include any introductory phrases, personal comments, or conversational language. 
    Provide only the segmented chapters followed by the bullet points in the output."""

    # Split the document into parts if it exceeds the model's capacity
    max_length = 2000  # Example: maximum number of characters per part
    document_parts = split_document(document_file_path, max_length)

    all_transcriptions = []
    chapter_counter = 1  # Initialize chapter counter

    # Process each part
    for part in document_parts:
        # Create a temporary file for the part
        temp_file_path = "temp_document_part.txt"
        with open(temp_file_path, 'w', encoding='utf-8') as temp_file:  # Specify encoding
            temp_file.write(part)

        # Upload the document part
        document_file = genai.upload_file(path=temp_file_path, display_name=os.path.basename(temp_file_path))
        print(f"Document part-{len(all_transcriptions) + 1} uploaded successfully.")  # Output part index

        # Wait for a moment to ensure the file is in an active state
        time.sleep(30)  # Increase the wait time if necessary

        # Retry logic for generating transcription
        for attempt in range(3):  # Try up to 3 times
            try:
                # Generate transcription
                response = model.generate_content([
                    prompt,
                    document_file
                ])
                print("Transcription response received for part:", len(all_transcriptions) + 1)  # Output part index

                # Update chapter numbering in the response text
                response_text = response.text

                # Find all chapters in the response and update their numbering
                chapters = re.findall(r'Chapter \d+', response_text)
                for chapter in chapters:
                    current_chapter_number = int(chapter.split()[-1])
                    new_chapter_number = chapter_counter + current_chapter_number - 1
                    response_text = response_text.replace(chapter, f"Chapter {new_chapter_number}")

                chapter_counter += len(chapters)  # Increment chapter counter by the number of chapters found

                all_transcriptions.append(response_text)
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Attempt {attempt + 1}: Error during transcription for part {len(all_transcriptions) + 1}: {e}")
                time.sleep(5)  # Wait before retrying

    return "\n\n".join(all_transcriptions)  # Combine all transcriptions # Define the document file path
document_file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'samples', 'document.txt'))

# Example usage
transcription_result = transcribe_document_with_segmented_chapters(document_file_path)
if transcription_result:
    print("Transcription Result:\n", transcription_result)
else:
    print("Failed to transcribe document.")

# Define the results folder path
results_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'results', 'document file'))

# Create the results directory if it doesn't exist
os.makedirs(results_folder, exist_ok=True)

# Define the output file path for the segmented chapters of the document file
output_md_file_path = os.path.join(results_folder, 'document_segmented_chapters.md')

# Save the segmented chapters to the specified markdown file
if transcription_result:
    with open(output_md_file_path, 'w', encoding='utf-8') as f:  # Specify encoding
        f.write(transcription_result)
    print(f"Segmented chapters from document file saved to: {output_md_file_path}")
else:
    print("No transcription result to save.")

Document part-1 uploaded successfully.
Transcription response received for part: 1
Document part-2 uploaded successfully.
Transcription response received for part: 2
Document part-3 uploaded successfully.
Transcription response received for part: 3
Transcription Result:
 **Chapter 1: Whispers of the Past Bookstore**

*   The bookstore, "Whispers of the Past," is located in a bustling city.
*   The bookstore's atmosphere is characterized by the scent of aged paper and ink, and a feeling of timelessness.
*   The bookstore's exterior evokes a sense of history and inviting exploration.
*   Mr. Thompson, the bookstore owner, is a long-standing figure in the neighborhood.
*   Mr. Thompson meticulously arranges the books and enjoys interacting with customers.


**Chapter 2: Clara's Arrival**

*   Clara, a young woman seeking inspiration for her writing, moves to the city.
*   Clara enters "Whispers of the Past" on a rainy afternoon.
*   Clara immediately feels a strong connection to the books

In [1]:
import os
import time
import subprocess  # Import subprocess for running FFmpeg commands
from dotenv import load_dotenv
import google.generativeai as genai
import re

# Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")
# Configure the Gemini API
genai.configure(api_key=api_key)

# Function to split video into smaller segments using FFmpeg
def split_video_ffmpeg(video_file_path, max_duration):
    segment_path_template = "segment_%03d.mp4"
    command = [
        "ffmpeg",
        "-i", video_file_path,
        "-c", "copy",
        "-map", "0",
        "-segment_time", str(max_duration),
        "-f", "segment",
        segment_path_template
    ]
    subprocess.run(command)

    # Generate a list of segment file names
    segments = [segment_path_template % i for i in range(len(os.listdir('.')))]
    return segments

# Function to transcribe video with segmented chapters
def transcribe_video_with_segmented_chapters(video_file_path):
    model = genai.GenerativeModel('models/gemini-1.5-flash-8b')
    prompt = """Please transcribe this video file and segment the transcription into meaningful chapters. 
    For each chapter, provide 5-6 detailed bullet points that summarize the key ideas, themes, or events discussed in that chapter. 
    Do not include any introductory phrases, personal comments, or conversational language. 
    Provide only the segmented chapters followed by the bullet points in the output."""

    max_duration = 300  # Example: 5 minutes
    split_video_ffmpeg(video_file_path, max_duration)

    all_transcriptions = []
    chapter_counter = 1  # Initialize chapter counter

    # Process each segment
    for segment in os.listdir('.'):
        if segment.startswith("segment_") and segment.endswith(".mp4"):
            video_file = genai.upload_file(path=segment, display_name=os.path.basename(segment))
            print(f"Video segment {segment} uploaded successfully.")

            time.sleep(30)

            for attempt in range(3):
                try:
                    response = model.generate_content([prompt, video_file])
                    print("Transcription response received for segment:", segment)

                    # Update chapter numbering in the response text
                    response_text = response.text

                    # Find all chapters in the response and update their numbering
                    chapters = re.findall(r'Chapter \d+', response_text)
                    for chapter in chapters:
                        current_chapter_number = int(chapter.split()[-1])
                        new_chapter_number = chapter_counter + current_chapter_number - 1
                        response_text = response_text.replace(chapter, f"Chapter {new_chapter_number}")

                    chapter_counter += len(chapters)  # Increment chapter counter by the number of chapters found

                    all_transcriptions.append(response_text)
                    break  # Exit the retry loop if successful
                except Exception as e:
                    print(f"Attempt {attempt + 1}: Error during transcription for segment {segment}: {e}")
                    time.sleep(5)

    return "\n\n".join(all_transcriptions)

# Define the video file path
video_file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'samples', 'video.mp4'))

# Example usage
transcription_result = transcribe_video_with_segmented_chapters(video_file_path)
if transcription_result:
    print("Transcription Result:\n", transcription_result)
else:
    print("Failed to transcribe video.")

# Define the results folder path
results_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'results', 'video file'))

# Create the results directory if it doesn't exist
os.makedirs(results_folder, exist_ok=True)

# Define the output file path for the segmented chapters of the video file
output_md_file_path = os.path.join(results_folder, 'video_segmented_chapters.md')

# Save the segmented chapters to the specified markdown file
if transcription_result:
    with open(output_md_file_path, 'w') as f:
        f.write(transcription_result)
    print(f"Segmented chapters from video file saved to: {output_md_file_path}")
else:
    print("No transcription result to save.")

Video segment segment_000.mp4 uploaded successfully.
Transcription response received for segment: segment_000.mp4
Video segment segment_001.mp4 uploaded successfully.
Transcription response received for segment: segment_001.mp4
Video segment segment_002.mp4 uploaded successfully.
Transcription response received for segment: segment_002.mp4
Video segment segment_003.mp4 uploaded successfully.
Transcription response received for segment: segment_003.mp4
Transcription Result:
 Chapter 1

* Desmond Doss was a conscientious objector in World War II.
* He was a medic who refused to carry a weapon.
* His religious beliefs were central to his decision.
* Seventh-day Adventism was the guiding force behind his convictions.
* Non-violence and a healthy, vegetarian diet were also key aspects of his beliefs.
* His actions showcased his strong moral compass.


Chapter 2

* Doss's refusal to carry a weapon led to conflict and opposition from his fellow soldiers.
* His superiors tried to pressure him 

In [7]:
import os
import time
from pydub import AudioSegment  # Import AudioSegment from pydub
from dotenv import load_dotenv
import google.generativeai as genai
import re

# Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")
# Configure the Gemini API
genai.configure(api_key=api_key)

# Function to split audio into smaller segments using pydub
def split_audio(audio_file_path, max_duration):
    audio = AudioSegment.from_file(audio_file_path)
    duration = len(audio)  # Duration in milliseconds
    segments = []

    for start in range(0, duration, max_duration * 1000):  # Convert max_duration to milliseconds
        end = min(start + max_duration * 1000, duration)
        segment = audio[start:end]
        segment_path = f"segment_{start // 1000}_{end // 1000}.mp3"  # Naming segments in seconds
        segment.export(segment_path, format="mp3")
        segments.append(segment_path)

    return segments

# Function to transcribe audio with segmented chapters
def transcribe_audio_with_segmented_chapters(audio_file_path):
    model = genai.GenerativeModel('models/gemini-1.5-flash-8b')
    prompt = """Please transcribe this audio file and segment the transcription into meaningful chapters. 
    For each chapter, provide 5-6 detailed bullet points that summarize the key ideas, themes, or events discussed in that chapter. 
    Do not include any introductory phrases, personal comments, or conversational language. 
    Provide only the segmented chapters followed by the bullet points in the output."""

    max_duration = 300  # Example: 5 minutes
    audio_segments = split_audio(audio_file_path, max_duration)

    all_transcriptions = []
    chapter_counter = 1  # Initialize chapter counter

    # Process each segment
    for segment in audio_segments:
        audio_file = genai.upload_file(path=segment, display_name=os.path.basename(segment))
        print(f"Audio segment {segment} uploaded successfully.")

        time.sleep(30)

        for attempt in range(3):
            try:
                response = model.generate_content([prompt, audio_file])
                print("Transcription response received for segment:", segment)

                # Update chapter numbering in the response text
                response_text = response.text

                # Find all chapters in the response and update their numbering
                chapters = re.findall(r'Chapter \d+', response_text)
                for chapter in chapters:
                    current_chapter_number = int(chapter.split()[-1])
                    new_chapter_number = chapter_counter + current_chapter_number - 1
                    response_text = response_text.replace(chapter, f"Chapter {new_chapter_number}")

                chapter_counter += len(chapters)  # Increment chapter counter by the number of chapters found

                all_transcriptions.append(response_text)
                break  # Exit the retry loop if successful
            except Exception as e:
                print(f"Attempt {attempt + 1}: Error during transcription for segment {segment}: {e}")
                time.sleep(5)

    return "\n\n".join(all_transcriptions)

# Define the audio file path
audio_file_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'samples', 'audio.mp3'))

# Example usage
transcription_result = transcribe_audio_with_segmented_chapters(audio_file_path)
if transcription_result:
    print("Transcription Result:\n", transcription_result)
else:
    print("Failed to transcribe audio.")

# Define the results folder path
results_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'results', 'audio file'))

# Create the results directory if it doesn't exist
os.makedirs(results_folder, exist_ok=True)

# Define the output file path for the segmented chapters of the audio file
output_md_file_path = os.path.join(results_folder, 'audio_segmented_chapters.md')

# Save the segmented chapters to the specified markdown file
if transcription_result:
    with open(output_md_file_path, 'w') as f:
        f.write(transcription_result)
    print(f"Segmented chapters from audio file saved to: {output_md_file_path}")
else:
    print("No transcription result to save.")

Audio segment segment_0_300.mp3 uploaded successfully.
Transcription response received for segment: segment_0_300.mp3
Audio segment segment_300_600.mp3 uploaded successfully.
Transcription response received for segment: segment_300_600.mp3
Audio segment segment_600_619.mp3 uploaded successfully.
Transcription response received for segment: segment_600_619.mp3
Transcription Result:
 Chapter 1: War Crimes Against Medics

*   Social conventions and international law aim to limit deliberate targeting on the battlefield.
*   Despite these conventions, individuals deemed off-limits are sometimes intentionally targeted during combat.
*   Rules and customs of war have existed for centuries.
*   In the 18th century, the officer class in armies was largely made up of the aristocracy.
*   The desire to conduct war in a "gentlemanly" fashion often led to the avoidance of targeting officers.
*   The deliberate targeting of officers was considered barbaric.


Chapter 2: Targeting Officers During the

In [25]:
pip install yt-dlp

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 20.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install openai-whisper


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install uuid

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6508 sha256=41ede8471bf83ae62288d27ee8ad312ddb1570fe7756fd9781c845e73d64e9ba
  Stored in directory: c:\users\ant pc\appdata\local\pip\cache\wheels\35\34\36\b9f3546da107cf37bab75cdb3ce1ebd8d744648985d0111ca1
Successfully built uuid
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import time
import subprocess
import uuid
from dotenv import load_dotenv
import google.generativeai as genai
import whisper
import yt_dlp
from pydub import AudioSegment
import shutil

# Load environment variables from .env file
load_dotenv()

# Configure Gemini API
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

def download_video(video_url, output_dir):
    """Download video using yt-dlp with safe filename handling"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate safe filename with UUID
    unique_id = str(uuid.uuid4())[:8]
    output_path = os.path.join(output_dir, f"video_{unique_id}.mp4")
    
    ydl_opts = {
        'format': 'bestaudio/best',  # Prefer audio-only formats
        'outtmpl': output_path,
        'quiet': True,
        'no_warnings': True
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"Downloaded video from: {video_url}")
        return output_path
    except Exception as e:
        print(f"Error downloading video: {e}")
        raise

def convert_video_to_audio(video_path, audio_path):
    """Convert video to audio using ffmpeg"""
    try:
        print("Converting video to audio...")
        subprocess.run(['ffmpeg', '-i', video_path, '-vn', '-acodec', 'mp3', audio_path], check=True)
        print(f"Audio saved to: {audio_path}")
    except Exception as e:
        print(f"Error during audio conversion: {e}")
        raise

def split_audio(audio_path, segment_length_ms=60000):
    """Split the audio into parts"""
    audio = AudioSegment.from_mp3(audio_path)
    segments = []
    for i in range(0, len(audio), segment_length_ms):
        segment = audio[i:i+segment_length_ms]
        segment_path = f"{audio_path}_{i // segment_length_ms}.mp3"
        segment.export(segment_path, format="mp3")
        segments.append(segment_path)
    return segments

def transcribe_with_whisper(audio_path, language="hi"):
    """Transcribe audio using Whisper model"""
    print("Loading Whisper model...")
    model = whisper.load_model("base")  # You can use "small" or "medium" for better accuracy
    
    print("Transcribing audio...")
    result = model.transcribe(
        audio_path,
        language=language,  # Specify Hindi language
        task="transcribe",  # For transcription (use "translate" for direct English translation)
    )
    
    return result["text"]

def segment_transcription(transcription):
    """Use Gemini to segment transcription into meaningful chapters"""
    model = genai.GenerativeModel('models/gemini-1.5-flash-8b')
    
    prompt = f"""Given this transcription, segment it into meaningful chapters. For each chapter, provide 5-6 detailed bullet points that summarize the key points, themes, or events discussed.

Transcription:
{transcription}

Please format the output as follows for each chapter:

Chapter X
• [First key point]
• [Second key point]
• [Third key point]
• [Fourth key point]
• [Fifth key point]

Focus on main ideas and maintain a clear narrative flow between chapters. Do not include any introductory phrases or personal comments"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error during chapter segmentation: {e}")
        raise

def cleanup_files(file_paths):
    """Clean up temporary files"""
    for file_path in file_paths:
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error cleaning up file {file_path}: {e}")

def main():
    try:
        video_url = input("Enter the URL of the video (YouTube, Khan Academy, TED Talks, etc.): ")
        output_directory = "downloaded_videos"
        
        # Download video
        print("Downloading video...")
        video_path = download_video(video_url, output_directory)

        # Convert video to audio
        audio_path = os.path.join(output_directory, f"audio_{str(uuid.uuid4())[:8]}.mp3")
        convert_video_to_audio(video_path, audio_path)

        # Split audio into segments
        print("Splitting audio into segments...")
        audio_segments = split_audio(audio_path)

        # Transcribe each audio segment with Whisper
        transcriptions = []
        for segment in audio_segments:
            print(f"Transcribing {segment}...")
            transcription = transcribe_with_whisper(segment)
            transcriptions.append(transcription)

        # Combine transcriptions into a single string
        full_transcription = " ".join(transcriptions)

        # Segment using Gemini
        print("\nGenerating chaptered summary...")
        chaptered_content = segment_transcription(full_transcription)
        print("\nChaptered Content:\n")
        print(chaptered_content)
        
        # Define the results folder path
        results_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'results', 'youtube transcription'))
        
        # Create the results directory if it doesn't exist
        os.makedirs(results_folder, exist_ok=True)
        
        # Define the output file path for the segmented chapters of the audio file
        output_md_file_path = os.path.join(results_folder, 'youtube_segmented_chapters.md')
        
        if chaptered_content:
            with open(output_md_file_path, 'w') as f:
                f.write(chaptered_content)
            print(f"Segmented chapters from youtube video url saved to: {output_md_file_path}")
        else:
            print("No transcription result to save.")
        
        # Cleanup
        cleanup_files([video_path, audio_path] + audio_segments)
        
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()


Enter the URL of the video (YouTube, Khan Academy, TED Talks, etc.):  https://www.youtube.com/watch?v=UkkEZ1F0WwU


Downloaded video from: https://www.youtube.com/watch?v=UkkEZ1F0WwU
Converting video to audio...
Audio saved to: downloaded_videos\audio_6fc0ca77.mp3
Splitting audio into segments...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_0.mp3...
Loading Whisper model...


C:\Users\ANT pc\AppData\Roaming\Python\Python312\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locatio

Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_1.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_2.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_3.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_4.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_5.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_6.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_7.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_6fc0ca77.mp3_8.mp3...
Loading Whisper model...
Transcribing audio...

Generating chaptered summary...

Chaptered Content:

Chapter 1
• The speaker is reviewing the Vande Bharat train.
• The review fo

In [31]:
import os
import time
import subprocess
import uuid
from dotenv import load_dotenv
import google.generativeai as genai
import whisper
import yt_dlp
from pydub import AudioSegment
import shutil
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Ensure nltk resources are downloaded
nltk.download('punkt')

# Load environment variables from .env file
load_dotenv()

# Configure Gemini API
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

def download_video(video_url, output_dir):
    """Download video using yt-dlp with safe filename handling"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate safe filename with UUID
    unique_id = str(uuid.uuid4())[:8]
    output_path = os.path.join(output_dir, f"video_{unique_id}.mp4")
    
    ydl_opts = {
        'format': 'bestaudio/best',  # Prefer audio-only formats
        'outtmpl': output_path,
        'quiet': True,
        'no_warnings': True
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"Downloaded video from: {video_url}")
        return output_path
    except Exception as e:
        print(f"Error downloading video: {e}")
        raise

def convert_video_to_audio(video_path, audio_path):
    """Convert video to audio using ffmpeg"""
    try:
        print("Converting video to audio...")
        subprocess.run(['ffmpeg', '-i', video_path, '-vn', '-acodec', 'mp3', audio_path], check=True)
        print(f"Audio saved to: {audio_path}")
    except Exception as e:
        print(f"Error during audio conversion: {e}")
        raise

def split_audio(audio_path, segment_length_ms=60000):
    """Split the audio into parts"""
    audio = AudioSegment.from_mp3(audio_path)
    segments = []
    for i in range(0, len(audio), segment_length_ms):
        segment = audio[i:i+segment_length_ms]
        segment_path = f"{audio_path}_{i // segment_length_ms}.mp3"
        segment.export(segment_path, format="mp3")
        segments.append(segment_path)
    return segments

def transcribe_with_whisper(audio_path, language="hi"):
    """Transcribe audio using Whisper model"""
    print("Loading Whisper model...")
    model = whisper.load_model("base")  # You can use "small" or "medium" for better accuracy
    
    print("Transcribing audio...")
    result = model.transcribe(
        audio_path,
        language=language,  # Specify Hindi language
        task="transcribe",  # For transcription (use "translate" for direct English translation)
    )
    
    return result["text"]

def segment_transcription(transcription):
    """Use Gemini to segment transcription into meaningful chapters"""
    model = genai.GenerativeModel('models/gemini-1.5-flash-8b')
    
    prompt = f"""Given this transcription, segment it into meaningful chapters. For each chapter, provide 5-6 detailed bullet points that summarize the key points, themes, or events discussed.

Transcription:
{transcription}

Please format the output as follows for each chapter:

Chapter X
• [First key point]
• [Second key point]
• [Third key point]
• [Fourth key point]
• [Fifth key point]

Focus on main ideas and maintain a clear narrative flow between chapters. Do not include any introductory phrases or personal comments"""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error during chapter segmentation: {e}")
        raise

embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def compute_semantic_consistency(full_transcription, chaptered_content):
    """Calculate semantic consistency using cosine similarity between full transcription and chapter summaries"""
    full_transcription_embedding = embedding_model.encode([full_transcription])
    
    # Split chaptered content into individual chapters
    chapters = chaptered_content.split("\n\n")
    
    chapter_scores = []
    
    for chapter in chapters:
        # Extract chapter summary (skip chapter title, get bullet points)
        lines = chapter.strip().split("\n")
        chapter_summary = " ".join(lines[1:])  # Skip "Chapter X" title
        chapter_embedding = embedding_model.encode([chapter_summary])
        
        # Compute cosine similarity between the full transcription and chapter summary
        similarity_score = cosine_similarity(full_transcription_embedding, chapter_embedding)[0][0]
        chapter_scores.append(similarity_score)
    
    avg_semantic_consistency = np.mean(chapter_scores)
    return avg_semantic_consistency, chapter_scores

def compute_relevance(chaptered_content, relevant_keywords):
    """Calculate relevance of each chapter based on keyword matching"""
    chapters = chaptered_content.split("\n\n")
    
    relevance_scores = []
    
    for chapter in chapters:
        # Extract chapter summary (skip chapter title, get bullet points)
        lines = chapter.strip().split("\n")
        chapter_summary = " ".join(lines[1:])
        
        # Tokenize the chapter summary and relevant keywords
        chapter_tokens = word_tokenize(chapter_summary.lower())
        relevant_tokens = [word.lower() for word in relevant_keywords]
        
        # Count keyword occurrences in chapter
        chapter_token_counts = Counter(chapter_tokens)
        relevance_score = sum(chapter_token_counts.get(keyword, 0) for keyword in relevant_tokens)
        
        relevance_scores.append(relevance_score)
    
    avg_relevance_score = np.mean(relevance_scores)
    return avg_relevance_score, relevance_scores

def evaluate_segmentation(full_transcription, chaptered_content, relevant_keywords):
    """Evaluate segmentation using semantic consistency and relevance metrics"""
    print("Evaluating segmentation quality...")

    # Compute semantic consistency
    avg_semantic_consistency, chapterwise_semantic_scores = compute_semantic_consistency(full_transcription, chaptered_content)
    print(f"Average Semantic Consistency: {avg_semantic_consistency:.4f}")
    for i, score in enumerate(chapterwise_semantic_scores):
        print(f"Chapter {i+1} Semantic Consistency: {score:.4f}")
    
    # Compute relevance
    avg_relevance_score, chapterwise_relevance_scores = compute_relevance(chaptered_content, relevant_keywords)
    print(f"Average Relevance Score: {avg_relevance_score:.4f}")
    for i, score in enumerate(chapterwise_relevance_scores):
        print(f"Chapter {i+1} Relevance Score: {score}")

    return avg_semantic_consistency, avg_relevance_score


def cleanup_files(file_paths):
    """Clean up temporary files"""
    for file_path in file_paths:
        try:
            if os.path.exists(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error cleaning up file {file_path}: {e}")


def main():
    try:
        video_url = input("Enter the URL of the video (YouTube, Khan Academy, TED Talks, etc.): ")
        output_directory = "downloaded_videos"
        
        # Download video
        print("Downloading video...")
        video_path = download_video(video_url, output_directory)

        # Convert video to audio
        audio_path = os.path.join(output_directory, f"audio_{str(uuid.uuid4())[:8]}.mp3")
        convert_video_to_audio(video_path, audio_path)

        # Split audio into segments
        print("Splitting audio into segments...")
        audio_segments = split_audio(audio_path)

        # Transcribe each audio segment with Whisper
        transcriptions = []
        for segment in audio_segments:
            print(f"Transcribing {segment}...")
            transcription = transcribe_with_whisper(segment)
            transcriptions.append(transcription)

        # Combine transcriptions into a single string
        full_transcription = " ".join(transcriptions)

        # Segment using Gemini
        print("\nGenerating chaptered summary...")
        chaptered_content = segment_transcription(full_transcription)
        print("\nChaptered Content:\n")
        print(chaptered_content)

        # Define relevant keywords (you can adjust based on the content or use a more sophisticated method)
        relevant_keywords = ["bomber", "America", "Hawaii", "training", "Boeing", "Fortress", "Lancaster", "Avro", "Flying"]

        # Evaluate segmentation quality
        evaluate_segmentation(full_transcription, chaptered_content, relevant_keywords)
        
        # Define the results folder path
        results_folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'results', 'youtube transcription'))
        
        # Create the results directory if it doesn't exist
        os.makedirs(results_folder, exist_ok=True)
        
        # Define the output file path for the segmented chapters of the audio file
        output_md_file_path = os.path.join(results_folder, 'youtube_segmented_chapters.md')
        
        if chaptered_content:
            with open(output_md_file_path, 'w') as f:
                f.write(chaptered_content)
            print(f"Segmented chapters from youtube video url saved to: {output_md_file_path}")
        else:
            print("No transcription result to save.")
        
        # Cleanup
        cleanup_files([video_path, audio_path] + audio_segments)
        
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to C:\Users\ANT
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Enter the URL of the video (YouTube, Khan Academy, TED Talks, etc.):  https://www.youtube.com/watch?v=YShmfsVCcZ4


Downloaded video from: https://www.youtube.com/watch?v=YShmfsVCcZ4
Converting video to audio...
Audio saved to: downloaded_videos\audio_3e47ee8f.mp3
Splitting audio into segments...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_0.mp3...
Loading Whisper model...


C:\Users\ANT pc\AppData\Roaming\Python\Python312\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locatio

Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_1.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_2.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_3.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_4.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_5.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_6.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_7.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_8.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_videos\audio_3e47ee8f.mp3_9.mp3...
Loading Whisper model...
Transcribing audio...
Transcribing downloaded_